# MapReduce на mrjob

mrjob это библиотека для Python, которая позволяет создавать MapReduce jobs. 

Документация по MRJob: [https://mrjob.readthedocs.io/en/latest/](https://mrjob.readthedocs.io/en/latest/)

Возьмем файл из MapReduce(bash) `fruits.txt`

In [1]:
!pip install mrjob

     |████████████████████████████████| 439 kB 1.5 MB/s eta 0:00:01
You should consider upgrading via the '/Users/pro/anaconda/bin/python -m pip install --upgrade pip' command.


In [2]:
%%bash
ls -lh fruits.txt

-rw-r--r--  1 pro  staff    59B  1 мар 10:25 fruits.txt


Создадим файл `word_count.py` с помощью magic в Jupyter Notebook `%%file`

In [1]:
%%file word_count.py
from mrjob.job import MRJob

class MRWordFrequencyCount(MRJob):

    def mapper(self, _, line):
        # когда используем yield, функция не останавливается в отличие от return 
        # yield работает как генератор 
        yield "chars", len(line)
        yield "words", len(line.split())
        yield "lines", 1
    # на каждый ключ суммируем
    def reducer(self, key, values):
        yield key, sum(values)


if __name__ == '__main__':
    MRWordFrequencyCount.run()

Overwriting word_count.py


In [2]:
!cat fruits.txt

apple banana
peach orange peach peach
pineapple peach apple

In [3]:
!cat fruits.txt | python word_count.py

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /var/folders/b1/01k3115d6txccqktf5wpvqdr0000gn/T/word_count.pro.20210301.093022.610760
Running step 1 of 1...
reading from STDIN
job output is in /var/folders/b1/01k3115d6txccqktf5wpvqdr0000gn/T/word_count.pro.20210301.093022.610760/output
Streaming final output from /var/folders/b1/01k3115d6txccqktf5wpvqdr0000gn/T/word_count.pro.20210301.093022.610760/output...
"words"	9
"lines"	3
"chars"	57
Removing temp directory /var/folders/b1/01k3115d6txccqktf5wpvqdr0000gn/T/word_count.pro.20210301.093022.610760...


Мы можем определит кол-во мапперов и редьюсеров, на пример $10$ мапперов и $3$ редьюсеров. (игрушечный пример)

In [1]:
%%bash
# запускаем локально
DATAFILE=fruits.txt
STREAMING_JAR=/opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar
N=10

# N map tasks
# запускаем локально (в питоне)
python word_count.py\
    # количество маперов
    --jobconf mapreduce.job.maps=$N\
    # количество редьюсеров
    --jobconf mapreduce.job.reduces=3\
    -r hadoop\
    --hadoop-streaming-jar $STREAMING_JAR\
    $DATAFILE

"words"	12
"lines"	6
"chars"	170


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /var/folders/b1/01k3115d6txccqktf5wpvqdr0000gn/T/word_count.pro.20210303.184935.772454
Running step 1 of 1...
reading from STDIN
job output is in /var/folders/b1/01k3115d6txccqktf5wpvqdr0000gn/T/word_count.pro.20210303.184935.772454/output
Streaming final output from /var/folders/b1/01k3115d6txccqktf5wpvqdr0000gn/T/word_count.pro.20210303.184935.772454/output...
Removing temp directory /var/folders/b1/01k3115d6txccqktf5wpvqdr0000gn/T/word_count.pro.20210303.184935.772454...


Проведем эксперименты

In [ ]:
%%bash
START=$(date +%s);

DATAFILE=# файл с большим набором данных
STREAMING_JAR=/opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar
N=4 # кол-во мапперов

# N map tasks
python word_count.py\
    --jobconf mapreduce.job.maps=$N\
    --jobconf mapreduce.job.reduces=3\
    -r hadoop --hadoop-streaming-jar $STREAMING_JAR\
    $DATAFILE
    
2>/dev/null

END=$(date +%s);
echo $((END-START)) | awk '{print "Duration: "int($1/60)":"int($1%60)}'